In [12]:
import fiona
import os
from ops.ops import load_json, create_exps_paths, load_exp
from tqdm.notebook import tqdm_notebook
from osgeo import gdal
from osgeo import gdalconst
import numpy as np
from tensorflow.keras.utils import to_categorical
from skimage.morphology import area_opening


In [13]:
conf = load_json(os.path.join('conf', 'conf.json'))
img_source = conf['img_source']
batch_size = conf['batch_size']
learning_rate = conf['learning_rate']
n_train_models = conf['n_train_models']
patch_size = conf['patch_size']
n_classes = conf['n_classes']
n_opt_layers = conf['n_opt_layers']
n_sar_layers = conf['n_sar_layers']
class_weights = conf['class_weights']
test_crop = conf['test_crop']
n_imgs = conf['n_imgs']

#exp_name = 'sm_opt_pm'
exp_name = 'rs_sm_opt_pm_nc_newstrategy'

exp_path = os.path.join('exps', exp_name)
models_path = os.path.join(exp_path, 'models')
logs_path = os.path.join(exp_path, 'logs')
pred_path = os.path.join(exp_path, 'predicted')
visual_path = os.path.join(exp_path, 'visual')


prep_path = os.path.join('img', 'prepared')

## Create Geotiff Predictions Probabilities

In [14]:
for test_image in range(n_imgs):
    pred = np.load(os.path.join(pred_path, f'pred_{test_image}.npy'))
    label = np.load(os.path.join(prep_path, f'label_2019.npy'))
    pred_p = pred[:,:,1:2]
    pred_p[label==2] = 0
    in_data = gdal.Open( os.path.join('img', 'labels', f'label_2019.tif'), gdalconst.GA_ReadOnly)
    geo_transform = in_data.GetGeoTransform()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * in_data.RasterXSize
    y_min = y_max + geo_transform[5] * in_data.RasterYSize
    x_res = in_data.RasterXSize
    y_res = in_data.RasterYSize
    crs = in_data.GetSpatialRef()
    proj = in_data.GetProjection()

    output = os.path.join(visual_path, f'pred_prob_{exp_name}_{test_image}.tif')
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Float32)
    
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetSpatialRef(crs)
    target_ds.SetProjection(proj)
    
    band = target_ds.GetRasterBand(1)
    band.WriteArray(pred_p[:,:,0], 0, 0)
    
    band.FlushCache()
    target_ds = None

FileNotFoundError: [Errno 2] No such file or directory: 'exps\\rs_sm_opt_pm_nc_newstrategy\\predicted\\pred_1.npy'

## Create Geotiff Cloud Predictions

In [11]:
for test_image in range(n_imgs):
    pred = np.load(os.path.join(pred_path, f'pred_{test_image}.npy'))
    label = np.load(os.path.join(prep_path, f'label_2019.npy'))
    pred_p = (pred[:,:,1:2]>0.5).astype(np.uint8)
    pred_p[label==2] = 0
    pred_p = area_opening(pred_p.squeeze(), 625)

    in_data = gdal.Open( os.path.join('img', 'labels', f'label_2019.tif'), gdalconst.GA_ReadOnly)
    geo_transform = in_data.GetGeoTransform()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * in_data.RasterXSize
    y_min = y_max + geo_transform[5] * in_data.RasterYSize
    x_res = in_data.RasterXSize
    y_res = in_data.RasterYSize
    crs = in_data.GetSpatialRef()
    proj = in_data.GetProjection()

    output = os.path.join(visual_path, f'pred_bin_{exp_name}_{test_image}.tif')
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
    
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetSpatialRef(crs)
    target_ds.SetProjection(proj)
    
    band = target_ds.GetRasterBand(1)
    band.WriteArray(pred_p, 0, 0)
    
    band.FlushCache()
    target_ds = None

FileNotFoundError: [Errno 2] No such file or directory: 'exps\\rs_sm_opt_pm_nc_comp\\predicted\\pred_1.npy'

## Create Geotiff Binary Predictions

In [5]:
with fiona.open(grid_save) as grid:
    for feat in tqdm_notebook(grid, desc = 'Tiles evaluated'):
        if feat['properties']['dataset'] !=2:
            continue
        feat_id = int(feat['properties']['id'])
        
        pred = np.load(os.path.join(predictions_path, f'{feat_id}.npy'))
        fz = np.load(os.path.join(test_tiles_path, f'{feat_id}.npz'))
        label = fz['label']
        pred_b = np.zeros_like(label)
        pred_b[pred[:,:,1]>0.5] = 1
        pred_b[label == 2] = 0
        pred_b = area_opening(pred_b.squeeze(), 625)
        
        
        
        in_data = gdal.Open( os.path.join('img', 'labels', f'label_{feat_id}.tif'), gdalconst.GA_ReadOnly)
        geo_transform = in_data.GetGeoTransform()
        x_min = geo_transform[0]
        y_max = geo_transform[3]
        x_max = x_min + geo_transform[1] * in_data.RasterXSize
        y_min = y_max + geo_transform[5] * in_data.RasterYSize
        x_res = in_data.RasterXSize
        y_res = in_data.RasterYSize
        crs = in_data.GetSpatialRef()
        proj = in_data.GetProjection()
        
        output = os.path.join(visual_path, f'{feat_id}_binary_pred_binary_exp_{exp_n}.tif')
        target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
        
        target_ds.SetGeoTransform(geo_transform)
        target_ds.SetSpatialRef(crs)
        target_ds.SetProjection(proj)
        
        band = target_ds.GetRasterBand(1)
        band.WriteArray(pred_b, 0, 0)
        
        band.FlushCache()
        target_ds = None


Tiles evaluated:   0%|          | 0/30 [00:00<?, ?it/s]

In [20]:
pred_b.shape

(2896, 2895, 1)

## Create Geotiff error binary

In [21]:
with fiona.open(grid_save) as grid:
    for feat in tqdm_notebook(grid, desc = 'Tiles evaluated'):
        if feat['properties']['dataset'] !=2:
            continue
        feat_id = int(feat['properties']['id'])
        
        pred = np.load(os.path.join(predictions_path, f'{feat_id}.npy'))
        fz = np.load(os.path.join(test_tiles_path, f'{feat_id}.npz'))
        label = fz['label']
        pred_b = np.zeros_like(label)
        pred_b[pred[:,:,1]>0.5] = 1
        pred_b[label == 2] = 0
        pred_b = area_opening(pred_b.squeeze(), 625)
        pred_b = np.expand_dims(pred_b, axis=-1)
        
        error_b = np.zeros_like(label)
        error_b[label==2] = 1 #background
        error_b[np.logical_and(label==1, pred_b==0)] = 2 #FN
        error_b[np.logical_and(label==0, pred_b==1)] = 3 #FP
        error_b[np.logical_and(label==1, pred_b==1)] = 4 #TP
        error_b[np.logical_and(label==0, pred_b==0)] = 5 #TN
        
        
        
        in_data = gdal.Open( os.path.join('img', 'labels', f'label_{feat_id}.tif'), gdalconst.GA_ReadOnly)
        geo_transform = in_data.GetGeoTransform()
        x_min = geo_transform[0]
        y_max = geo_transform[3]
        x_max = x_min + geo_transform[1] * in_data.RasterXSize
        y_min = y_max + geo_transform[5] * in_data.RasterYSize
        x_res = in_data.RasterXSize
        y_res = in_data.RasterYSize
        crs = in_data.GetSpatialRef()
        proj = in_data.GetProjection()
        
        output = os.path.join(visual_path, f'{feat_id}_binary_error_b_exp_{exp_n}.tif')
        target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
        
        target_ds.SetGeoTransform(geo_transform)
        target_ds.SetSpatialRef(crs)
        target_ds.SetProjection(proj)
        
        band = target_ds.GetRasterBand(1)
        band.WriteArray(error_b[:,:,0], 0, 0)
        
        band.FlushCache()
        target_ds = None


Tiles evaluated:   0%|          | 0/30 [00:00<?, ?it/s]

## Generate minimum error map

In [10]:
opt_exp = 1
sar_exp = 2
fus_exp = 3


output_path = os.path.join('exps', 'compile')

with fiona.open(grid_save) as grid:
    for feat in tqdm_notebook(grid, desc = 'Tiles evaluated'):
        if feat['properties']['dataset'] !=2:
            continue
        feat_id = int(feat['properties']['id'])

        opt_pred = np.load(os.path.join('exps', f'exp_{opt_exp}', 'results', 'predictions', f'{feat_id}.npy'))[:,:,1:2]
        sar_pred = np.load(os.path.join('exps', f'exp_{sar_exp}', 'results', 'predictions', f'{feat_id}.npy'))[:,:,1:2]
        fus_pred = np.load(os.path.join('exps', f'exp_{fus_exp}', 'results', 'predictions', f'{feat_id}.npy'))[:,:,1:2]
        fz = np.load(os.path.join(test_tiles_path, f'{feat_id}.npz'))
        label = fz['label']

        error_min = np.zeros_like(label)
        gt = label ==1
        opt_error = np.abs(opt_pred - gt)
        sar_error = np.abs(sar_pred - gt)
        fus_error = np.abs(fus_pred - gt)

        errors = np.concatenate([opt_error, sar_error, fus_error], axis=2)
        errors_arg = np.expand_dims(np.argmin(errors, axis=2), axis=-1)+1
        errors_arg[label==2] = 0


        in_data = gdal.Open( os.path.join('img', 'labels', f'label_{feat_id}.tif'), gdalconst.GA_ReadOnly)
        geo_transform = in_data.GetGeoTransform()
        x_min = geo_transform[0]
        y_max = geo_transform[3]
        x_max = x_min + geo_transform[1] * in_data.RasterXSize
        y_min = y_max + geo_transform[5] * in_data.RasterYSize
        x_res = in_data.RasterXSize
        y_res = in_data.RasterYSize
        crs = in_data.GetSpatialRef()
        proj = in_data.GetProjection()
        
        output = os.path.join(output_path, f'{feat_id}_error.tif')
        target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
        
        target_ds.SetGeoTransform(geo_transform)
        target_ds.SetSpatialRef(crs)
        target_ds.SetProjection(proj)
        
        band = target_ds.GetRasterBand(1)
        band.WriteArray(np.squeeze(errors_arg), 0, 0)
        
        band.FlushCache()
        target_ds = None




Tiles evaluated:   0%|          | 0/30 [00:00<?, ?it/s]